In [ ]:
# For understanding, how pyspark SQL operates, we use 
# https://www.codementor.io/jadianes/python-spark-sql-dataframes-du107w74i
# for help
# it worked once, so we could read the 'sheet.csv'-file which contains 36 rows of the typical taxidata
# due to massive problems with the python kernel on juptyer, it was not possible to replicate the results for some screenshots

from pyspark.sql import SQLContext
from pyspark.sql import Row
from pyspark.streaming.kafka import KafkaUtils
from pyspark.streaming import StreamingContext
from pyspark import SparkContext, SparkConf

config = SparkConf().setAppName('test-topic')
sc = SparkContext(conf=config)
#ssc = StreamingContext("local[2]", 1) # create streaming context
#raw_data = KafkaUtils.createDirectStream(ssc, ['test-topic'], { "metadata.broker.list": "localhost:9092"})

sqlContext = SQLContext(sc)
#extr_data = raw_data.map(lambda fu: fu[1])
#extr_data.pprint(20)
#print(extr_data)
extr_data = sc.read.csv("hdfs://loudacre/sheet.csv").cache()
#extr_data.show()
csv_data = extr_data.map(lambda l: l.split(",")).zipWithIndex().filter(lambda (row,index): index > 1).keys()
print(csv_dataghhgj7865,
      ,
row_data = csv_data.map(lambda p: Row(
        VendorID=int(p[0]), 
        tpep_pickup_datetime=p[1],
        tpep_dropoff_datetime=p[2],
        passenger_count=int(p[3]),
        trip_distance=float(p[4]),
        ratecodeID=int(p[5]),
        store_and_fwd_flag=str(p[6]),
        pulocationid=int(p[7]),
        dolocationid=int(p[8]),
        payment_type=int(p[9]),
        fare_amount=float(p[10]),
        extra=float(p[11]),
        mta_tax=float(p[12]),
        tip_amount=float(p[13]),
        tolls_amount=float(p[14]),
        improvement_surcharge=float(p[15]),
        total_amount=float(p[16])
    )
)

taxi_df= sqlContext.createDataFrame(row_data)
taxi_df.registerTempTable("taxi")

taxi_test= sqlContext.sql("""
    SELECT pulocationid, avg(tip_amount) as Average_tip_amount FROM taxi WHERE payment_type = '1' GROUP BY pulocationid
""")

taxi_test.show()

#ssc.start()
#ssc.awaitTermination()
#ssc.stop()

In [ ]:
#just some shitty
import sys
!sudo pyspark --packages com.databricks:spark-csv_2.10:1.3.

In [ ]:
# our different approach to read the data via Spark, but there we couldn't load the package 'com.databricks.spark.csv'
# https://github.com/databricks/spark-csv as a reference
# spark-shell --packages com.databricks:spark-csv_2.1xxxx
# and 
# spark-submit --packages com.databricks:spark-csv_2.1 didn't work out, like manual says.

from pyspark.sql import SQLContext
from pyspark.sql.types import *

sqlContext = SQLContext(sc)
customSchema = StructType([ \
    StructField("VendorID", IntegerType(), True), \
    StructField("tpep_pickup_datetime", StringType(), True), \
    StructField("tpep_dropoff_datetime", StringType(), True), \
    StructField("passenger_count", IntegerType(), True), \
    StructField("trip_distance", FloatType(), True), \
    StructField("RatecodeID", IntegerType(), True), \
    StructField("store_and_fwd_flag", StringType(), True), \
    StructField("PULocationID", IntegerType(), True), \
    StructField("DOLocationID", IntegerType(), True), \
    StructField("payment_type", IntegerType(), True), \
    StructField("fare_amount", FloatType(), True), \
    StructField("extra", FloatType(), True), \
    StructField("mta_tax", FloatType(), True), \
    StructField("tip_amount", FloatType(), True), \
    StructField("tolls_amount", FloatType(), True), \
    StructField("improvement_surcharge", FloatType(), True), \
    StructField("total_amount", FloatType(), True)])

df = sqlContext.read \
    .format('com.databricks.spark.csv') \
    .options(header='true') \
    .load('Desktop/sheet.csv', schema = customSchema)

df.select('PULocationID', 'tip_amount', 'payment_type').write \
    .format('com.databricks.spark.csv') \
    .save('Desktop/was.csv')

In [ ]:
import os
SUBMIT_ARGS = "--packages com.databricks:spark-csv_2.11:1.2.0 pyspark-shell"
os.environ["PYSPARK_SUBMIT_ARGS"] = SUBMIT_ARGS

In [ ]:
export PYTHONPATH=$SPARK_HOME/python/lib/py4j-0.8.2.1-src.zip:$SPARK_HOME/python/:$PYTHONPATH

In [ ]:
!whereis python2.7

In [ ]:
import kafka
from kafka import SimpleProducer, SimpleClient
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

TOPIC = "test-topic"

sc = SparkContext("local[2]","Streamer")
ssc = StreamingContext(sc, 1)

# To send messages synchronously
client = SimpleClient('localhost:9092')
producer = SimpleProducer(client, async=False)

# Note that the application is responsible for encoding messages to type bytes
producer.send_messages(TOPIC, '1,2017-01-09 11:13:28,2017-01-09 11:25:45,1,3.30,1,N,263,161,1,12.5,0,0.5,2,0,0.3,15.3')


ssc.start()             # Start the computation
ssc.awaitTermination()

In [ ]:
import sys
!sudo {sys.executable} -m pip install py4j